In [1]:
import spacy

nlp = spacy.load("en_core_web_sm")

In [2]:
sentences = [
  'The food we had yesterday was delicious',
  'My time in Italy was very enjoyable',
  'I found the meal to be tasty',
  'The internet was slow.',
  'Our experience was suboptimal'
]

In [9]:
# For each word in the sentence, check all its dependencies

for sentence in sentences:
  doc = nlp(sentence)
  for token in doc:
    print(token.text,'-1-', token.dep_,'-2-',  token.head.text,'-3-',  token.head.pos_,'-4-', token.pos_,'-5-', [child for child in token.children])

The -1- det -2- food -3- NOUN -4- DET -5- []
food -1- nsubj -2- was -3- AUX -4- NOUN -5- [The, had]
we -1- nsubj -2- had -3- AUX -4- PRON -5- []
had -1- relcl -2- food -3- NOUN -4- AUX -5- [we, yesterday]
yesterday -1- npadvmod -2- had -3- AUX -4- NOUN -5- []
was -1- ROOT -2- was -3- AUX -4- AUX -5- [food, delicious]
delicious -1- acomp -2- was -3- AUX -4- ADJ -5- []
My -1- poss -2- time -3- NOUN -4- DET -5- []
time -1- nsubj -2- was -3- AUX -4- NOUN -5- [My, in]
in -1- prep -2- time -3- NOUN -4- ADP -5- [Italy]
Italy -1- pobj -2- in -3- ADP -4- PROPN -5- []
was -1- ROOT -2- was -3- AUX -4- AUX -5- [time, enjoyable]
very -1- advmod -2- enjoyable -3- ADJ -4- ADV -5- []
enjoyable -1- acomp -2- was -3- AUX -4- ADJ -5- [very]
I -1- nsubj -2- found -3- VERB -4- PRON -5- []
found -1- ROOT -2- found -3- VERB -4- VERB -5- [I, be]
the -1- det -2- meal -3- NOUN -4- DET -5- []
meal -1- nsubj -2- be -3- AUX -4- NOUN -5- [the]
to -1- aux -2- be -3- AUX -4- PART -5- []
be -1- ccomp -2- found -3- VER

In [10]:
# Pick out all the adjectives

for sentence in sentences:
  doc = nlp(sentence)
  descriptive_term = ''
  for token in doc:
    if token.pos_ == 'ADJ':
      descriptive_term = token
  print(sentence)
  print(descriptive_term)

The food we had yesterday was delicious
delicious
My time in Italy was very enjoyable
enjoyable
I found the meal to be tasty
tasty
The internet was slow.
slow
Our experience was suboptimal
suboptimal


In [11]:
# Pick out all the adverbs for the adjectives

for sentence in sentences:
  doc = nlp(sentence)
  descriptive_term = ''
  for token in doc:
    if token.pos_ == 'ADJ':
      prepend = ''
      for child in token.children:
        if child.pos_ != 'ADV':
          continue
        prepend += child.text + ' '
      descriptive_term = prepend + token.text
  print(sentence)
  print(descriptive_term)

The food we had yesterday was delicious
delicious
My time in Italy was very enjoyable
very enjoyable
I found the meal to be tasty
tasty
The internet was slow.
slow
Our experience was suboptimal
suboptimal


In [12]:
# We will now identify the targets and its descriptions

aspects = []
for sentence in sentences:
  doc = nlp(sentence)
  descriptive_term = ''
  target = ''
  for token in doc:
    if token.dep_ == 'nsubj' and token.pos_ == 'NOUN':
      target = token.text
    if token.pos_ == 'ADJ':
      prepend = ''
      for child in token.children:
        if child.pos_ != 'ADV':
          continue
        prepend += child.text + ' '
      descriptive_term = prepend + token.text
  aspects.append({'aspect': target,
    'description': descriptive_term})
print(aspects)

[{'aspect': 'food', 'description': 'delicious'}, {'aspect': 'time', 'description': 'very enjoyable'}, {'aspect': 'meal', 'description': 'tasty'}, {'aspect': 'internet', 'description': 'slow'}, {'aspect': 'experience', 'description': 'suboptimal'}]


In [13]:

from textblob import TextBlob
for aspect in aspects:
  aspect['sentiment'] = TextBlob(aspect['description']).sentiment
print(aspects)

[{'aspect': 'food', 'description': 'delicious', 'sentiment': Sentiment(polarity=1.0, subjectivity=1.0)}, {'aspect': 'time', 'description': 'very enjoyable', 'sentiment': Sentiment(polarity=0.65, subjectivity=0.78)}, {'aspect': 'meal', 'description': 'tasty', 'sentiment': Sentiment(polarity=0.0, subjectivity=0.0)}, {'aspect': 'internet', 'description': 'slow', 'sentiment': Sentiment(polarity=-0.30000000000000004, subjectivity=0.39999999999999997)}, {'aspect': 'experience', 'description': 'suboptimal', 'sentiment': Sentiment(polarity=0.0, subjectivity=0.0)}]


In [15]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [20]:
# To improve the above algorithm, we can train a Naive Bayes Calssifier in TextBlob

from textblob.classifiers import NaiveBayesClassifier
# We train the NaivesBayesClassifier
train = [
  ('Slow internet.', 'negative'),
  ('Delicious food', 'positive'),
  ('Suboptimal experience', 'negative'),
  ('Very enjoyable time', 'positive'),
  ('delicious food.', 'neg')
]
cl = NaiveBayesClassifier(train)
# And then we try to classify some sample sentences.
blob = TextBlob("Delicious food. Very Slow internet. Suboptimal experience. Boring experience. Enjoyable food.", classifier=cl)
for s in blob.sentences:
  print(s)
  print(s.classify())

Delicious food.
positive
Very Slow internet.
negative
Suboptimal experience.
negative
Boring experience.
negative
Enjoyable food.
positive
